In [1]:
import argparse
import ast
import itertools
import pathlib
import warnings

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import toml
import tqdm
from joblib import dump
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import ElasticNetCV, LogisticRegression, MultiTaskElasticNetCV

# import RepeatedKFold
from sklearn.model_selection import (
    GridSearchCV,
    LeaveOneOut,
    RepeatedKFold,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.utils import parallel_backend

In [2]:
# argparser = argparse.ArgumentParser()
# argparser.add_argument("--cell_type", type=str, default="all")
# argparser.add_argument("--shuffle", type=str, default=False)
# argparser.add_argument("--cytokine", type=str, default="cytokine")
# argparser.add_argument("--feature_columns", type=str, default="all")

# args = argparser.parse_args()

# cell_type = args.cell_type
# shuffle = args.shuffle
# cytokine = args.cytokine
# feature_columns = args.feature_columns


cell_type = "PBMC"
cytokine = "IL-1 beta [NSU]"
shuffle = "False"
feature_columns = []

print(cell_type, shuffle, cytokine)
if shuffle == "True":
    shuffle = True
elif shuffle == "False":
    shuffle = False
else:
    raise ValueError("shuffle must be True or False")
print(f"shuffle: {shuffle}")

PBMC False IL-1 beta [NSU]
shuffle: False


In [3]:
aggregation = True
nomic = True

In [4]:
# set shuffle value
if shuffle:
    shuffle = "shuffled_baseline"
else:
    shuffle = "final"

In [5]:
MODEL_TYPE = "regression"

In [6]:
# load training data from indexes and features dataframe
data_split_path = pathlib.Path(
    f"../../0.split_data/indexes/{cell_type}/regression/aggregated_sc_and_nomic_data_split_indexes.tsv"
)
feature_combinations_path = pathlib.Path(
    f"../../0.split_data/results/feature_combinations_{cell_type}.toml"
).resolve()
data_path = pathlib.Path(
    f"../../../data/{cell_type}_preprocessed_sc_norm_aggregated_nomic.parquet"
)

# dataframe with only the labeled data we want (exclude certain phenotypic classes)
data_df = pd.read_parquet(data_path)
data_df = data_df[columns]

data_split_indexes = pd.read_csv(data_split_path, sep="\t")

(154, 405)

In [7]:
# select tht indexes for the training and test set
train_indexes = data_split_indexes.loc[data_split_indexes["label"] == "train"]
# subset data_df by indexes in data_split_indexes
training_data = data_df.loc[train_indexes["labeled_data_index"]]
# define metadata columns
# subset each column that contains metadata
metadata = training_data.filter(regex="Metadata")
# drop all metadata columns
data_x = training_data.drop(metadata.columns, axis=1)
labeled_data = training_data["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
# get all columns that contain "NSU" in the column name
data_y_cols = data_x.filter(regex="NSU").columns
train_y = training_data[data_y_cols]
train_x = data_x.drop(data_y_cols, axis=1)
train_x = train_x.drop(columns="oneb_Treatment_Dose_Inhibitor_Dose")
loo = LeaveOneOut()
loo.get_n_splits(train_y)

train_data_y = train_y[cytokine]
model = ElasticNetCV(
    random_state=0,
    max_iter=10000,
    cv=loo,
    l1_ratio=[0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 0.99],
    alphas=[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    fit_intercept=True,
    selection="random",
)
# train model on training data on all combinations of model types, feature types, and phenotypic classes

In [8]:
if shuffle == "shuffled_baseline":
    print("Shuffling data")
    for column in train_x:
        np.random.shuffle(train_x[column].values)
else:
    print("Not shuffling data")
# define parameters to search over
with parallel_backend("multiprocessing"):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
        # create a logistic regression model
        model.fit(train_x, train_data_y)
        scores = cross_val_score(
            model,
            train_x,
            train_data_y,
            scoring="neg_mean_absolute_error",
            cv=loo,
            n_jobs=-1,
        )
        print(scores)
        print(f"Mean MAE: {scores.mean()}")
        print(f"Std MAE: {scores.std()}")
        print(f"R2: {model.score(train_x, train_data_y)}")

if (aggregation == True) and (nomic == True):
    results_dir = f"../models/regression/{cell_type}/aggregated_with_nomic/"
elif (aggregation == True) and (nomic == False):
    results_dir = f"../models/regression/{cell_type}/aggregated/"
elif (aggregation == False) and (nomic == True):
    results_dir = f"../models/regression/{cell_type}/sc_with_nomic/"
elif (aggregation == False) and (nomic == False):
    results_dir = f"../models/regression/{cell_type}/sc/"
else:
    print("Error")

# create results directory if it doesn't exist
pathlib.Path(results_dir).mkdir(parents=True, exist_ok=True)

# save final estimator
if shuffle == "shuffled_baseline":
    dump(
        model,
        f"{results_dir}/{cytokine}_shuffled_baseline__all_nomic.joblib",
    )
elif shuffle == "final":
    dump(
        model,
        f"{results_dir}/{cytokine}_final__all_nomic.joblib",
    )
else:
    print("Error")

# save condfig copy specific to this model to the folder with the results
# use pathlib
if shuffle == "shuffled_baseline":
    config_copy_path = pathlib.Path(
        f"{results_dir}/{cytokine}_shuffled_baseline__all_nomic.toml"
    )
elif shuffle == "final":
    config_copy_path = pathlib.Path(f"{results_dir}/{cytokine}_final__all_nomic.toml")
else:
    print("Error")

# write toml file with parameters used from injected parameters

with open(config_copy_path, "w") as f:
    f.write(f"model_type='{shuffle}'\n")
    f.write(f"aggregation={aggregation}\n")
    f.write(f"nomic={nomic}\n")
    f.write(f"cell_type='{cell_type}'\n")
    f.write(f"feature=all\n")

Not shuffling data
[-0.02609314 -0.00956301 -0.10737673 -0.26352596 -0.07130847 -0.20497906
 -0.04740904 -0.13977031 -0.13269473 -0.02839714 -0.13077255 -0.08117413
 -0.20453965 -0.01705748 -0.06921506 -0.03684608 -0.43450018 -0.09996979
 -0.09671219 -0.18096532 -0.3797989  -0.02854044 -0.15257703 -0.0684652
 -0.2072909  -0.01386964 -0.00554606 -0.02123841 -0.05075731 -0.08764562
 -0.06916677 -0.02059786 -0.02413939 -0.20129069 -0.14158451 -0.1542889
 -0.01212531 -0.22253498 -0.04319538 -0.27808273 -0.00496623 -0.04517764
 -0.10009938 -0.26872233 -0.00126308 -0.07989805 -0.19277696 -0.34975074
 -0.06442987 -0.06282666 -0.08991878 -0.20100688 -0.11528072 -0.12282024
 -0.30326235 -0.03210391 -0.20333693 -0.0847217  -0.16908793 -0.25815482
 -0.22093541 -0.43614344 -0.08744676 -0.02926864 -0.35056141 -0.25905717
 -0.18415928 -0.17411629 -0.13463256 -0.07711892 -0.04870537]
Mean MAE: -0.13123037321680836
Std MAE: 0.1064962316491161
R2: 0.9070818525802957
